<a href="https://colab.research.google.com/github/ximeeb/ebertz-gross-lopez-tp-lcs/blob/master/malignant_vs_benign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets
!pip install pandas

In [2]:
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from pandas.io.formats.info import DataFrameTableBuilder
import opendatasets as od
import pandas
import cv2
import os
import matplotlib.pyplot as plt
import json
from PIL import Image

In [ ]:
od.download("https://www.kaggle.com/datasets/fanconic/skin-cancer-malignant-vs-benign")

In [5]:
def cargar_imagenes(ruta_carpeta, etiqueta):
    archivos_img = os.listdir(ruta_carpeta)

    imagenes = []
    etiquetas = []
    tamano_img = 100

    for archivo_img in archivos_img:
        if es_imagen(archivo_img):
            ruta_imagen = os.path.join(ruta_carpeta, archivo_img)

            imagen = leer_imagen(ruta_imagen, tamano_img)

            imagenes.append(imagen)
            etiquetas.append(etiqueta)

    return imagenes, etiquetas

In [6]:
def es_imagen(archivo):
  return archivo.endswith('.jpg') or archivo.endswith('.png')

In [7]:
def leer_imagen(ruta, tamano):
  imagen = cv2.imread(ruta, cv2.COLOR_BGR2RGB)
  imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
  imagen = np.array(cv2.resize(imagen, (tamano, tamano)))

  return imagen

In [8]:
import random
def mezclar_con_correspondencia(imagenes, etiquetas):
  combinados = list(zip(imagenes, etiquetas))
  random.shuffle(combinados)
  i, e = zip(*combinados)
  return i, e

In [9]:
# Preparación de imágenes y etiquetas de entrenamiento

ruta_benign = "/content/skin-cancer-malignant-vs-benign/train/benign"
ruta_malignant = "/content/skin-cancer-malignant-vs-benign/train/malignant"

imagenes_benign, etiquetas_benign = cargar_imagenes(ruta_benign, 0)
imagenes_malignant, etiquetas_malignant = cargar_imagenes(ruta_malignant, 1)

imagenes_entrenamiento = imagenes_benign + imagenes_malignant
etiquetas_entrenamiento = etiquetas_benign + etiquetas_malignant

imagenes_entrenamiento, etiquetas_entrenamiento = mezclar_con_correspondencia(imagenes_entrenamiento, etiquetas_entrenamiento)

In [10]:
# Preparación de imágenes y etiquetas de prueba

ruta_benign = "/content/skin-cancer-malignant-vs-benign/test/benign"
ruta_malignant = "/content/skin-cancer-malignant-vs-benign/test/malignant"

imagenes_benign_prueba, etiquetas_benign_prueba = cargar_imagenes(ruta_benign, 0)
imagenes_malignant_prueba, etiquetas_malignant_prueba = cargar_imagenes(ruta_malignant, 1)

imagenes_prueba = imagenes_benign_prueba + imagenes_malignant_prueba
etiquetas_prueba = etiquetas_benign_prueba + etiquetas_malignant_prueba

imagenes_prueba, etiquetas_prueba = mezclar_con_correspondencia(imagenes_prueba, etiquetas_prueba)

In [ ]:
# Mostrar algunas imágenes

plt.figure(figsize = (10, 10))

for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(imagenes_entrenamiento[i])

plt.show()

In [11]:
# Normalización de entradas y convirtiendo listas a tensores

imagenes_entrenamiento = np.array(imagenes_entrenamiento).astype(float) / 255
imagenes_prueba = np.array(imagenes_prueba).astype(float) / 255

etiquetas_entrenamiento = np.array(etiquetas_entrenamiento)
etiquetas_prueba = np.array(etiquetas_prueba)

In [31]:
# Preparación de generador de datos

generador_datos = ImageDataGenerator(
    rotation_range = 50, # Rotación aleatoria de las imágenes
    width_shift_range = 0.1, # Movimiento hacia los lados
    height_shift_range = 0.1, # Movimiento hacia arriba o abajo
    shear_range = 15, # Inclinación
    zoom_range = [0.9, 1.0],
    horizontal_flip = True,
    vertical_flip = True
)

generador_datos.fit(imagenes_entrenamiento)

In [25]:
modelo_denso = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (100, 100, 3)),
    tf.keras.layers.Dense(150, activation = 'relu'),
    tf.keras.layers.Dense(150, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid'),
])

#Compilación
modelo_denso.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [34]:
modelo_cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (100, 100, 3)),
    tf.keras.layers.MaxPooling2D(3, 3),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

#Compilación
modelo_cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [26]:
modelo_cnn_ad = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(150, activation = 'relu'),
    tf.keras.layers.Dense(150, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

#Compilación
modelo_cnn_ad.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

generador_datos_entrenamiento = generador_datos.flow(imagenes_entrenamiento, etiquetas_entrenamiento, batch_size = 32)

In [15]:
epocas = 20

In [ ]:
print("Entrenando modelo denso...")
historial_denso = modelo_denso.fit(imagenes_entrenamiento, etiquetas_entrenamiento, epochs=epocas, validation_data=(imagenes_prueba, etiquetas_prueba), use_multiprocessing=True, shuffle=True)

print("Modelo denso entrenado!")

In [ ]:
print("Entrenando modelo convolucional...")
historial_cnn = modelo_cnn.fit(imagenes_entrenamiento, etiquetas_entrenamiento, epochs=epocas, validation_data=(imagenes_prueba, etiquetas_prueba), use_multiprocessing=True, shuffle=True)

print("Modelo convolucional entrenado!")

In [ ]:
print("Entrenando modelo convolucional con aumento de datos...")
historial_cnn_ad = modelo_cnn_ad.fit(
    generador_datos_entrenamiento,
    epochs=epocas,
    validation_data=(imagenes_prueba, etiquetas_prueba),
)

print("Modelo convolucional entrenado!")

In [ ]:
# Graficas de precisión y pérdida de modelo
acc = historial_cnn_ad.history['accuracy']
val_acc = historial_cnn_ad.history['val_accuracy']

loss = historial_cnn_ad.history['loss']
val_loss = historial_cnn_ad.history['val_loss']

rango_epocas = range(epocas)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(rango_epocas, acc, label='Precisión Entrenamiento')
plt.plot(rango_epocas, val_acc, label='Precisión Pruebas')
plt.legend(loc='lower right')
plt.ylim(0.4, 1)
plt.title('Precisión de entrenamiento y pruebas')

plt.subplot(1,2,2)
plt.plot(rango_epocas, loss, label= 'Pérdida de entrenamiento')
plt.plot(rango_epocas, val_loss, label='Pérdida de pruebas')
plt.legend(loc='upper right')
plt.ylim(0, 1)
plt.title('Pérdida de entrenamiento y pruebas')
plt.show()

In [ ]:
# Graficas de precisión y pérdida de todos los modelos
acc_denso = historial_denso.history['accuracy']
acc_cnn = historial_cnn.history['accuracy']
acc_cnn_ad = historial_cnn_ad.history['accuracy']

loss_denso = historial_denso.history['loss']
loss_cnn = historial_cnn.history['loss']
loss_cnn_ad = historial_cnn_ad.history['loss']

rango_epocas = range(epocas)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(rango_epocas, acc_denso, label='Modelo Denso')
plt.plot(rango_epocas, acc_cnn, label='Modelo CNN')
plt.plot(rango_epocas, acc_cnn_ad, label='Modelo CNN AD')
plt.legend(loc='lower right')
plt.ylim(0.4, 1)
plt.title('Precisión de entrenamiento')

plt.subplot(1,2,2)
plt.plot(rango_epocas, loss_denso, label='Modelo Denso')
plt.plot(rango_epocas, loss_cnn, label='Modelo CNN')
plt.plot(rango_epocas, loss_cnn_ad, label='Modelo CNN AD')
plt.legend(loc='upper right')
plt.ylim(0, 1)
plt.title('Pérdida de entrenamiento')
plt.show()

In [39]:
import math
def prediccion_es_correcta(esperado, prediccion):
    return round(prediccion[0]) == esperado

In [40]:
def imprimir_testing(predicciones, etiquetas_esperadas, cant_imagenes):
    correctas_tipo = [0, 0]
    falladas_tipo = [0, 0]

    print("Predicciones:", len(predicciones))

    for i in range(cant_imagenes):
      prediccion = predicciones[i]
      esperado = etiquetas_esperadas[i]

      if prediccion_es_correcta(esperado, prediccion):
        correctas_tipo[esperado] += 1
      else:
        falladas_tipo[esperado] += 1

    print("Tipo 0:", correctas_tipo[0], "correctas,", falladas_tipo[0], "incorrectas")
    print("Tipo 1:", correctas_tipo[1], "correctas,", falladas_tipo[1], "incorrectas")

    total_correctas = correctas_tipo[0] + correctas_tipo[1]
    print("Total:", total_correctas, "correctas,", cant_imagenes - total_correctas, "incorrectas\n")

In [ ]:
# Testing

imagenes = np.array(imagenes_prueba)

print("Predicciones del modelo denso")
imprimir_testing(modelo_denso.predict(imagenes), etiquetas_prueba, len(imagenes))

print("Predicciones del modelo convolutivo")
imprimir_testing(modelo_cnn.predict(imagenes), etiquetas_prueba, len(imagenes))

print("Predicciones del modelo convolutivo con aumento de datos")
imprimir_testing(modelo_cnn_ad.predict(imagenes), etiquetas_prueba, len(imagenes))